In [11]:
import os, sys
# import numpy as np
from functools import partial
import autograd.numpy as np
from autograd import jacobian, grad, primitive
from scipy.stats import multivariate_normal
from itertools import tee
import copy

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MomentMatching.StateModels import GaussianState
from MomentMatching.baseMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform
from MomentMatching.auto_grad import logpdf
from MomentMatching.ExpectationPropagation import TimeSeriesNodeForEP, EPbase, EPNodes

In [4]:
TT = TaylorTransform(dimension_of_state=1)

In [5]:
All_nodes = EPNodes(dimension_of_state=1, N=15)

In [ ]:
nodes = All_nodes.filter_iter()

In [ ]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [ ]:
pairs = pairwise(All_nodes)

In [ ]:
next(pairs)

In [7]:
def f(x):
    return 2*x + 1

In [23]:
def fwd_update(transform, transition_function, prev_node, node ):
    assert isinstance(node, TimeSeriesNodeForEP)
    assert isinstance(node.marginal, GaussianState)
    print(node.forward_factor)
    assert isinstance(node.forward_factor, GaussianState)
    forward_cavity = node.marginal / node.forward_factor
    back_cavity = prev_node.marginal / prev_node.back_factor
    pred_mean, pred_cov, pred_cross_cov = transform.predict(nonlinear_func=transition_function,
                                   distribution=back_cavity)
    
#     logZi, dlogZidMz, dlogZidSz =transform.project(nonlinear_func=transition_function,
#                                                    distribution=back_cavity,
#                                                    data=None)
    pred_state = GaussianState(mean_vec=pred_mean, cov_matrix=pred_cov)
    
    result_node = node
    forward_factor = GaussianState(mean_vec=pred_mean, cov_matrix=pred_cov)
    result_node.forward_factor = forward_factor
#     assert isinstance(other, GaussianState)
    result_node.marginal = forward_cavity * result_node.forward_factor
    
    return result_node
    

In [26]:
ep_node = fwd_update(transform=TT, transition_function=f, prev_node=All_nodes[2], node=All_nodes[3])

GaussianState 
 mean=
 [ 1.], 
 cov=
[[-44439.5062277]])


In [30]:
print(ep_node.forward_factor)

GaussianState 
 mean=
 [ 1.], 
 cov=
[[-44439.5062277]])


In [ ]:
for node1, node2 in nodes:
    print(f'node {node1.t}, node {node2.t}')